# Using the Script-Language Container

A [Script-Language Container](https://github.com/exasol/script-languages-release) for the Exasol database consists of a Linux container with a complete Linux distribution and all required libraries, such as a script client. The script client is responsible for the communication with the database and for executing the script code. It allows to also include user specific libraries which can then be used from within the UDF's.

## Preparing the Notebook

First we need to install and import a few Python packages required in the course of this Notebook. Luckily, all dependencies are already installed in the VM.

In [1]:
import bash_runner as bash # A helper to run bash with interactive output from python
import importlib
from pathlib import Path
import pyexasol
import requests
import textwrap

The [Script-Language Container](https://github.com/exasol/script-languages-release) is also already cloned in the home directory.


In [2]:
slc_path="$HOME/script-languages-release"


## Buiding and Exporting a Container

To build and export the container you can use `exaslct`. It first builds a series of Docker images and then exports the container as a `tar.gz` package. We provide several flavors of containers with different capabilities. You can find out more about the flavors in our [flavor documentation on Github.](https://github.com/exasol/script-languages-release/blob/master/flavors/README.md). Flavors are described by a flavor definition in the directory `flavors/`. Here is an overview of the available flavors:

In [3]:
bash.run(f"""
find {slc_path}/flavors/  -maxdepth 1 -name '*EXASOL*'
""")

/home/ubuntu/script-languages-release/flavors/python-3.7-minimal-EXASOL-6.2.0
/home/ubuntu/script-languages-release/flavors/standard-EXASOL-8.0.0
/home/ubuntu/script-languages-release/flavors/standard-EXASOL-7.0.0
/home/ubuntu/script-languages-release/flavors/r-4-minimal-EXASOL-6.2.0
/home/ubuntu/script-languages-release/flavors/python-3.6-minimal-EXASOL-6.2.0
/home/ubuntu/script-languages-release/flavors/python-3.8-minimal-EXASOL-6.2.0
/home/ubuntu/script-languages-release/flavors/standard-EXASOL-7.1.0
/home/ubuntu/script-languages-release/flavors/standard-EXASOL-7.1.0-without-python2.7


For this example, we use the `flavors/python-3.8-minimal-EXASOL-6.2.0` flavor and export it to the `containers` directory.

A container gets built via a series of Docker images, and then it's exported into a `tar` file.

In [4]:
bash.run(f"""
pushd {slc_path}
./exaslct export --flavor-path flavors/python-3.8-minimal-EXASOL-6.2.0 --export-path containers
""")

~/script-languages-release ~/notebooks
WARNING - AnalyzeFlavorCustomization_0bd81678ef(job_id=2022_09_22_18_43_33_ExportContainers, no_cache=False, flavor_path=flavors/python-3.8-minimal-EXASOL-6.2.0): Image exasol/script-language-container:python-3.8-minimal-EXASOL-6.2.0-flavor_customization_JNLUKSGDSD6WMNGLJJIVCXRVV46ULM4QSG36KCKFAP4SADAZ6UJQ not in registry, got exception 404 Client Error for http+docker://localhost/v1.41/images/create?tag=python-3.8-minimal-EXASOL-6.2.0-flavor_customization_JNLUKSGDSD6WMNGLJJIVCXRVV46ULM4QSG36KCKFAP4SADAZ6UJQ&fromImage=exasol%2Fscript-language-container: Not Found ("manifest for exasol/script-language-container:python-3.8-minimal-EXASOL-6.2.0-flavor_customization_JNLUKSGDSD6WMNGLJJIVCXRVV46ULM4QSG36KCKFAP4SADAZ6UJQ not found: manifest unknown: manifest unknown")
WARNING - AnalyzeRelease_0bd81678ef(job_id=2022_09_22_18_43_33_ExportContainers, no_cache=False, flavor_path=flavors/python-3.8-minimal-EXASOL-6.2.0): Image exasol/script-language-container

### What to do if something doesn't work?

During the build it can happen that external package repositories might not be available or something is wrong on your machine where you run the build. For these cases, `exaslct` stores many logs to identify the problem.

#### Exaslsct Log

The main log for `exaslct` is stored directly as `main.log` in the build output of the job. With the following command you can find the main logs for all previous executions.

In [5]:
bash.run(f"""
pushd {slc_path}
find .build_output -name 'main.log'
""")

~/script-languages-release ~/notebooks
.build_output/jobs/logs/main.log


With the following command you can show the log file from the last execution.

In [6]:
bash.run(f"""
pushd {slc_path}
LAST_LOG="$(find .build_output -name 'main.log' | sort |tail -n 1)"
cat $LAST_LOG | tail -n 20
""")

~/script-languages-release ~/notebooks
2022-09-22 18:49:12,255 - luigi-interface.AnalyzeRelease - DEBUG - AnalyzeRelease_0bd81678ef(job_id=2022_09_22_18_43_33_ExportContainers, no_cache=False, flavor_path=flavors/python-3.8-minimal-EXASOL-6.2.0): Cleaning up
2022-09-22 18:49:12,256 - luigi-interface.AnalyzeLanguageDeps - DEBUG - AnalyzeLanguageDeps_0bd81678ef(job_id=2022_09_22_18_43_33_ExportContainers, no_cache=False, flavor_path=flavors/python-3.8-minimal-EXASOL-6.2.0): Cleaning up
2022-09-22 18:49:12,257 - luigi-interface.AnalyzeUDFClientDeps - DEBUG - AnalyzeUDFClientDeps_0bd81678ef(job_id=2022_09_22_18_43_33_ExportContainers, no_cache=False, flavor_path=flavors/python-3.8-minimal-EXASOL-6.2.0): Cleaning up
2022-09-22 18:49:12,260 - luigi-interface.AnalyzeUDFClientDeps - DEBUG - AnalyzeUDFClientDeps_0bd81678ef(job_id=2022_09_22_18_43_33_ExportContainers, no_cache=False, flavor_path=flavors/python-3.8-minimal-EXASOL-6.2.0): Cleanup finished
2022-09-22 18:49:12,261 - luigi-interface.

#### Build Output Directory

More detailed information about the build or other operations can be found in the `.build_output/jobs/*/outputs` directory. Here each run of `exaslct` creates its own directory under `.build_output/jobs`. The `outputs` directory stores the outputs and log files (if any) that each executed task of `exaslct` produces. Especially, the Docker tasks such as build, pull and push store the logs returned by the Docker API. This can be helpful for finding problems during the build.

In [7]:
bash.run(f"""
pushd {slc_path}
find .build_output/jobs/*/outputs -type f -name '*log' | tail
""")

~/script-languages-release ~/notebooks
.build_output/jobs/2022_09_22_18_33_46_ExportContainers/outputs/ExportContainers_1a3be22a34/ExportFlavorContainer_be94de61b2/DockerCreateImageTask_5c09903eb8/DockerPullImageTask_5c09903eb8/logs/pull_docker_db_image.log
.build_output/jobs/2022_09_22_18_38_53_ExportContainers/outputs/ExportContainers_d73037b379/ExportFlavorContainer_f91b46c6bc/DockerCreateImageTask_0a7e2ec761/DockerPullImageTask_0a7e2ec761/logs/pull_docker_db_image.log
.build_output/jobs/2022_09_22_18_38_53_ExportContainers/outputs/ExportContainers_d73037b379/ExportFlavorContainer_f91b46c6bc/DockerCreateImageTask_31d9d5af1a/DockerPullImageTask_31d9d5af1a/logs/pull_docker_db_image.log
.build_output/jobs/2022_09_22_18_38_53_ExportContainers/outputs/ExportContainers_d73037b379/ExportFlavorContainer_f91b46c6bc/DockerCreateImageTaskWithDeps_f950681425/DockerBuildImageTask_1d37740f88/logs/docker-build.log
.build_output/jobs/2022_09_22_18_38_53_ExportContainers/outputs/ExportContainers_d73

## Customizing Script-Language Containers

Sometimes you need very specific dependencies or versions of dependencies in the Exasol UDFs. In such case you can customize a Script-Language Container.

### Flavor Definition

To customize a flavor, you need to change the flavor definition. A flavor consists of several build steps. The following images gives you an idea about how these build steps are connected with each other.

![Flavor Structure](slc_main_build_steps.svg)

For customizing a flavor usually the `flavor_customization` build step is most important. It contains everything you need to add dependencies. The remaining build steps should be only changed with care, but sometimes some dependencies are defined in other build steps because the script client depends on them. Here you see the directory structure of the flavor definition for our example flavor `flavors/python-3.8-minimal-EXASOL-6.2.0`.

In [8]:
bash.run(f""" 
pushd {slc_path}
find -L flavors/python-3.8-minimal-EXASOL-6.2.0 -maxdepth 2
""")

~/script-languages-release ~/notebooks
flavors/python-3.8-minimal-EXASOL-6.2.0
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/Dockerfile
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/packages
flavors/python-3.8-minimal-EXASOL-6.2.0/FLAVOR_DESCRIPTION.md
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/base_test_deps
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/build_run
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/build_steps.py
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/__pycache__
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/release
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/build_deps
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/base_test_build_run
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/security_scan
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_base/flavor_base_deps
flavors/python

### Flavor Customization Build Step

The `flavor_customization` build step consists of a Dockerfile and several package lists which can be modified. We recommend to use the package lists to add new packages to the flavor and only modify the Dockerfile if you need very specific changes, like adding additional resources.

In [9]:
bash.run(f""" 
pushd {slc_path}
find -L flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization -type f
""")

~/script-languages-release ~/notebooks
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/Dockerfile
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/packages/python3_pip_packages
flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/packages/apt_get_packages


#### Dockerfile

The Dockerfile consists of two parts. The first part installs the packages from the package list and should only be change with care. The second part is free for your changes. Read the description in the Dockerfile carefully to find out what you can and shouldn't do.

In [10]:
bash.run(f""" 
pushd {slc_path}
cat flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/Dockerfile
""")

~/script-languages-release ~/notebooks
############################################################################################
############################################################################################
# This Dockerfile allows you to extend this flavor by installing packages or adding files. 
# IF you didn't change the lines below, you can add packages and their version to the  
# files in ./packages and they get automatically installed.                                
############################################################################################
############################################################################################

#######################################################################
#######################################################################
# Do not change the following lines unless you know what you are doing 
#######################################################################
############################

#### Package Lists

The package lists have a unified format. Each line consists of the package name and the package version separated by "|", e.g `xgboost|1.3.3`. You can comment out a whole line by adding"#" at the beginning. You can also add a trailing comment to a package definition by adding a "#" after the package definition. We usually recommend to pin the version, such that there are no surprises for which version gets installed.

In [11]:
bash.run(f""" 
pushd {slc_path}
cat flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/packages/python3_pip_packages
""")

~/script-languages-release ~/notebooks
xgboost|1.3.3
scikit-learn|0.24.2


We are now going to append the "xgboost" Python package to one of the package lists by adding `"xgboost|1.3.3"` and `scikit-learn|0.24.2` to the `flavor_customization/packages/python3_pip_packages` file.

In [12]:
bash.run(f""" 
pushd {slc_path}
echo "xgboost|1.3.3" > flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/packages/python3_pip_packages
echo "scikit-learn|0.24.2" >> flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/packages/python3_pip_packages
""")

~/script-languages-release ~/notebooks


As you can see below, we now added a new line to our package list.

In [13]:
bash.run(f""" 
pushd {slc_path}
cat flavors/python-3.8-minimal-EXASOL-6.2.0/flavor_customization/packages/python3_pip_packages
""")

~/script-languages-release ~/notebooks
xgboost|1.3.3
scikit-learn|0.24.2


#### Rebuilding the customized Flavor

After changing the flavor you need to rebuild it. You can do it by running `./exaslsct export --flavor-path <flavor-path>` again. Exaslct automatically recognizes that the flavor has changed and builds a new version of the container.

In [14]:
bash.run(f"""
pushd {slc_path}
./exaslct export --flavor-path flavors/python-3.8-minimal-EXASOL-6.2.0 --export-path containers
""")

~/script-languages-release ~/notebooks
The command took 2.244718 s

Cached container under /home/ubuntu/script-languages-release/.build_output/cache/exports/python-3.8-minimal-EXASOL-6.2.0-release-NSFG3CPFKPLAHL4PLHHPUKG3UDIFENZCN2IC4J7QS6IQACRIKREQ.tar.gz

Copied container to containers/python-3.8-minimal-EXASOL-6.2.0_release.tar.gz




**Note:** Your old container doesn't get lost, because when you change a flavor your container gets a new hash code. If you revert your changes the system automatically uses the existing cached container. Below you can see the content of the cache directory for the containers.

In [15]:
bash.run(f"""
pushd {slc_path}
ls -sh .build_output/cache/exports
""")

~/script-languages-release ~/notebooks
total 611M
195M python-3.8-minimal-EXASOL-6.2.0-release-G2EN5EEXSTDQNKBBSUJ5JF6OVOKA7VCHHZQUVHMCR3YNKKLDQ62Q.tar.gz
4.0K python-3.8-minimal-EXASOL-6.2.0-release-G2EN5EEXSTDQNKBBSUJ5JF6OVOKA7VCHHZQUVHMCR3YNKKLDQ62Q.tar.gz.sha512sum
417M python-3.8-minimal-EXASOL-6.2.0-release-NSFG3CPFKPLAHL4PLHHPUKG3UDIFENZCN2IC4J7QS6IQACRIKREQ.tar.gz
4.0K python-3.8-minimal-EXASOL-6.2.0-release-NSFG3CPFKPLAHL4PLHHPUKG3UDIFENZCN2IC4J7QS6IQACRIKREQ.tar.gz.sha512sum


## Testing the new Script-Language Container

Now, that we have an updated container, we need to check if our changes were successful. For that we are going to upload the container to an Exasol Database and have a look into it. In this example, we are going to use a local Docker-DB started by `exaslct`, which uses our [integration-test-docker-environment](https://github.com/exasol/integration-test-docker-environment) in the background. 

**Note:** You could also use your own Exasol Database by changing the variables below. However, this Notebook must be able to access the BucketFS of your Exasol Database or you need to manually upload the container. 

In [16]:
DATABASE_HOST="localhost"
DATABASE_PORT=8886
DATABASE_USER="sys"
DATABASE_PASSWORD="exasol"
BUCKETFS_PORT=6666
BUCKETFS_USER="w"
BUCKETFS_PASSWORD="write"
BUCKETFS_NAME="bfsdefault"
BUCKET_NAME="default"
PATH_IN_BUCKET="container"

### Starting a local Docker-DB for Testing

The exaslct command also allows us to create rapidly a test environment where we can test the new container!

#### Start the environment and forward the database and bucketfs ports to the specified host ports.

**Note:** The Exasol Integration-Test-Docker-Environment requires Docker with privileged mode

**Note:** Starting the environment can take between 3-5 min.

In [17]:
bash.run(f"""
pushd {slc_path}
./exaslct spawn-test-environment --environment-name test --database-port-forward {DATABASE_PORT} --bucketfs-port-forward {BUCKETFS_PORT}
""")

~/script-languages-release ~/notebooks
WARNING - SpawnTestContainer_6d38778a73(job_id=2022_09_22_18_51_37_SpawnTestEnvironmentWithDockerDB, no_cache=False, environment_name=test, test_container_name=test_container_test, attempt=0): Copy tests in test container test_container_test.
WARNING - PopulateEngineSmallTestDataToDatabase_d164b7a3ea(job_id=2022_09_22_18_51_37_SpawnTestEnvironmentWithDockerDB, no_cache=False, db_user=sys, environment_name=test): Uploading data
The command took 286.471179 s


### Upload the Container to the Database

To use our container we need to upload it to the BucketFS . If the build machine has access to the BucketFS we can do it with the `exaslct upload` command, otherwise you need to export the container and transfer it to a machine that has access to the BucketFS and upload it via `curl`, as described in our [documentation](https://docs.exasol.com/database_concepts/udf_scripts/adding_new_packages_script_languages.htm).

With the following command we upload the new script language container to the previously created test environment.

In [18]:
bash.run(f"""
pushd {slc_path}
./exaslct upload \
    --flavor-path flavors/python-3.8-minimal-EXASOL-6.2.0 \
    --database-host {DATABASE_HOST}\
    --bucketfs-port {BUCKETFS_PORT} \
    --bucketfs-username {BUCKETFS_USER} \
    --bucketfs-password {BUCKETFS_PASSWORD} \
    --bucketfs-name {BUCKETFS_NAME} \
    --bucket-name {BUCKET_NAME} \
    --path-in-bucket {PATH_IN_BUCKET} \
    --release-name current
""")

~/script-languages-release ~/notebooks
The command took 84.426359 s

Uploaded .build_output/cache/exports/python-3.8-minimal-EXASOL-6.2.0-release-NSFG3CPFKPLAHL4PLHHPUKG3UDIFENZCN2IC4J7QS6IQACRIKREQ.tar.gz to
http://localhost:6666/default/container/python-3.8-minimal-EXASOL-6.2.0-release-current.tar.gz


In SQL, you can activate the languages supported by the python-3.8-minimal-EXASOL-6.2.0
flavor by using the following statements:


To activate the flavor only for the current session:

ALTER SESSION SET SCRIPT_LANGUAGES='PYTHON3=localzmq+protobuf:///bfsdefault/default/container/python-3.8-minimal-EXASOL-6.2.0-release-current?lang=python#buckets/bfsdefault/default/container/python-3.8-minimal-EXASOL-6.2.0-release-current/exaudf/exaudfclient_py3';


To activate the flavor on the system:

ALTER SYSTEM SET SCRIPT_LANGUAGES='PYTHON3=localzmq+protobuf:///bfsdefault/default/container/python-3.8-minimal-EXASOL-6.2.0-release-current?lang=python#buckets/bfsdefault/default/container/python-3.8-m

### Getting the language container activation statement without upload

Sometimes you can't use the `upload` command to upload your container to the BucketFS. To get the language activation statement regardless of that, you can use the `generate-language-activation` command.

In [ ]:
bash.run(f"""
pushd {slc_path}
./exaslct generate-language-activation \
    --flavor-path flavors/python-3.8-minimal-EXASOL-6.2.0 \
    --bucketfs-name {BUCKETFS_NAME} \
    --bucket-name {BUCKET_NAME} \
    --path-in-bucket {PATH_IN_BUCKET} \
    --container-name python-3.8-minimal-EXASOL-6.2.0-current  2>&1 | tail -n 15
""")

### Connecting to the database and activate the container

Once we have a connection to the database we run the `ALTER SESSION` statement or `ALTER SYSTEM` statement (if you want to activate the container permanently and globally) we got from the upload.

In [ ]:
def connect():
    con=pyexasol.connect(dsn=f"{DATABASE_HOST}:{DATABASE_PORT}",user=DATABASE_USER,password=DATABASE_PASSWORD)
    con.execute("ALTER SESSION SET SCRIPT_LANGUAGES='PYTHON3=builtin_python3 PYTHON3_DS=localzmq+protobuf:///bfsdefault/default/container/python-3.8-minimal-EXASOL-6.2.0-release-current?lang=python#buckets/bfsdefault/default/container/python-3.8-minimal-EXASOL-6.2.0-release-current/exaudf/exaudfclient_py3';")
    con.execute("OPEN SCHEMA TEST")
    return con

### Check if your customization did work

We first create a helper UDF which allows us to run arbitrary shell commands inside of a UDF instance. With that we can easily inspect the container.

In [ ]:
con = connect()

con.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3_DS SCALAR SCRIPT execute_shell_command_py3(command VARCHAR(2000000), split_output boolean)
EMITS (lines VARCHAR(2000000)) AS
import subprocess

def run(ctx):
    try:
        p = subprocess.Popen(ctx.command,
                             stdout    = subprocess.PIPE,
                             stderr    = subprocess.STDOUT,
                             close_fds = True,
                             shell     = True)
        out, err = p.communicate()
        if isinstance(out,bytes):
            out=out.decode('utf8')
        if ctx.split_output:
            for line in out.strip().split('\\n'):
                ctx.emit(line)
        else:
            ctx.emit(out)
    finally:
        if p is not None:
            try: p.kill()
            except: pass
/
"""))

#### Check with "pip list" if the  "xgboost" package is installed

We use our helper UDF to run `python3 -m pip list` directly in the container and get the list of currently avaiable python3 packages.

In [ ]:
con = connect()
rs=con.execute("""select execute_shell_command_py3('python3 -m pip list', true)""")
for r in rs: 
    print(r[0])

By running `pip list` directly in the container, we see what is currently available in the container. However, sometimes this might not be what we expected. For these cases, `exaslct` stores information about the flavor the container was build from within the container.

#### Embedded Build Information of the Container

Here we see an overview about the build information which `exaslct` embedded into the container. `Exaslct` stores all packages lists (as defined in the flavor and what actually got installed), the final Dockerfiles and the image info. The image info describes how the underlying Docker images of the container got built. The build information is stored in the `/build_info` directory in the container. We can use again our helper UDF to inspect the build information.

In [ ]:
con = connect()
rs=con.execute("""select execute_shell_command_py3('find /build_info', true)""")
for r in rs: 
    print(r[0])

The following command shows for example, which python3 package pip found directly after the build of the container image.

In [ ]:
con = connect()
rs=con.execute("""select execute_shell_command_py3('cat /build_info/actual_installed_packages/release/python3_pip_packages', true)""")
for r in rs: 
    print(r[0])

You could for example compare this to the package list of the `flavor-customization` build step and check if all your requested packages got installed.

In [ ]:
con = connect()
rs=con.execute("""select execute_shell_command_py3('cat /build_info/packages/flavor_customization/python3_pip_packages', true)""")
for r in rs:
    if r[0] is None:
        print()
    else:
        print(r[0])

### Testing the new package

After we made sure that the required packages are installed, we need to try importing and using it. Importing is usually a good first test if a package got successfully installed, because often you might already get errors at this step. However, sometimes you only will recognize errors when using the package. We recommend to have a test suite for each new package to check if it works properly before you start your UDF development. It is usually easier to debug problems if you have very narrow tests.

In [ ]:
con = connect()

con.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3_DS SET SCRIPT test_xgboost(i integer)
EMITS (o VARCHAR(2000000)) AS

def run(ctx):
    import xgboost
    import sklearn 
    
    ctx.emit("success")
/
"""))

rs = con.execute("select test_xgboost(1)")
rs.fetchall()

In [ ]:
con = connect()

con.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3_DS SET SCRIPT test_xgboost(i integer)
EMITS (o1 DOUbLE, o2 DOUbLE, o3 DOUbLE) AS

def run(ctx):
    import pandas as pd
    import xgboost as xgb
    from sklearn import datasets
    from sklearn.model_selection import train_test_split
    
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    param = {
        'max_depth': 3,  # the maximum depth of each tree
        'eta': 0.3,  # the training step for each iteration
        'silent': 1,  # logging mode - quiet
        'objective': 'multi:softprob',  # error evaluation for multiclass training
        'num_class': 3  # the number of classes that exist in this datset
        }
    num_round = 20  # the number of training iterations
    bst = xgb.train(param, dtrain, num_round)
    preds = bst.predict(dtest)
    
    ctx.emit(pd.DataFrame(preds))
/
"""))

con.export_to_pandas("select test_xgboost(1)")